In [ ]:
import requests
from API_KEY import BING_KEY
import shutil
import os
import matplotlib.pyplot as plt
import skimage.io
import numpy as np
from tqdm.autonotebook import tqdm

In [ ]:
STOREFOLDER = "data/bingcrawl"

In [ ]:
def create_dir_if_not_exist(directory):
    import os, errno
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

In [ ]:
IMG_SHAPE = "512,512"
ZOOM_LEVELS = [ 13, 15, 18, 20]
MAP_TYPE = ["Aerial", "Road"]
GERMANY = {'north': 53.385433, 'east': 14.394068,
           'west': 6.373989, 'south': 47.439950}

# test functions

In [ ]:
longitude = 49.428124
latitude = 8.643959846153846
maptype = "Road" 
#maptype = "Aerial"

In [ ]:
#URL = f"https://dev.virtualearth.net/REST/v1/Imagery/Map/{ maptype }/Redmond Washington/15?mapSize={ IMG_SHAPE }&key={ BING_KEY }&highlightEntity=1&dpi=Large&fmt=png&style={ STYLE }"

In [ ]:
STYLE = 'structure|visible:true;fillColor:FF0000_road|labelVisible:false'

In [ ]:
# documentation: https://msdn.microsoft.com/en-us/library/ff701724.aspx
# zoomLevel ( 0 ... 21 )

URL = f"https://dev.virtualearth.net/REST/v1/Imagery/Map/{ maptype }/{ longitude },{ latitude }/15?mapSize={ IMG_SHAPE }&key={ BING_KEY }&highlightEntity=1&dpi=Large&fmt=jpeg&style={ STYLE }"

In [ ]:
# testing basic functions of url
res = requests.get(URL, stream=True)
with open("test.jpg", "wb") as f:
    res.raw.decode_content = True
    shutil.copyfileobj(res.raw, f)
img = skimage.io.imread("test.jpg")
plt.imshow(img)

In [ ]:
FELD_LONG = 49.422879
FELD_LAT = 8.675183

In [ ]:
def store_map(long, lat, maptype = "Aerial", zoom = 15, show=False):
    style = "road|labelVisible:false_area|labelVisible:false"
    url = f"https://dev.virtualearth.net/REST/v1/Imagery/Map/{ maptype }/{ long },{ lat }/{ zoom }?mapSize={ IMG_SHAPE }&key={ BING_KEY }&highlightEntity=0&dpi=Large&fmt=jpeg&style={ style }"
    try:
        res = requests.get(url, stream=True)
    except ConnectionError:
        print("[WARNING] ran into rate-limit")
        import time
        time.sleep(1)
        try:
            res = requests.get(url, stream=True)
        except:
            return -1
        
        
    dir_nm = f"{ STOREFOLDER }/{ zoom }/{ maptype }"
    fn = f"{ dir_nm }/img_{ long }_{ lat }.jpeg"
    create_dir_if_not_exist(dir_nm)
    
    with open(fn, "wb") as f:
        res.raw.decode_content = True
        shutil.copyfileobj(res.raw, f)
    if show:
        img = skimage.io.imread(fn)
        plt.imshow(img)

In [ ]:
store_map(FELD_LONG, FELD_LAT)

In [ ]:
# create grid of HD images
# this could be a cool calculation depending on zoom level + overlapping percentage,
# but lets first use a count of wanted images as stupid heuristic
# Heidelberg:
WEST_NORTH = ( 49.428124, 8.630286 )
EAST_SOUTH = ( 49.405175, 8.719166 )

# Stuttgart:
WEST_NORTH = ( 48.809443, 9.130144 )
EAST_SOUTH = ( 48.751753, 9.260488 )


len_long = abs(WEST_NORTH[0] - EAST_SOUTH[0])
len_lat = abs(WEST_NORTH[1] - EAST_SOUTH[1])
print("LEN_LONG:", len_long, "LEN_LAT:", len_lat)
ratio_long = len_long/len_lat
print("Ratio_long:", ratio_long)

count = 1000

long_space = np.linspace(WEST_NORTH[0], EAST_SOUTH[0], int(np.sqrt(count))) #np.floor(ratio_long*count).astype(int))
lat_space = np.linspace(WEST_NORTH[1], EAST_SOUTH[1], int(np.sqrt(count))) #np.floor((1-ratio_long) * count).astype(int))

print(long_space, lat_space)

In [ ]:
for zoom in tqdm([15], desc="levels"):
    for maptype in tqdm(MAP_TYPE, desc="maptype", leave=False):
        for long in tqdm(long_space, desc="longitude", leave=False):
            for lat in tqdm(lat_space, desc="latitude", leave=False):
                store_map(long, lat, maptype=maptype, zoom=zoom)